# Template for segmented MNL models

Sam Maurer, Sep 2018

This notebook tests and demonstrates the segmented MNL models added in [UrbanSim Templates PR #45](https://github.com/UDST/urbansim_templates/pull/45).

In [1]:
import numpy as np
import pandas as pd

import orca
import urbansim_templates

from urbansim_templates import modelmanager
from urbansim_templates.models import SegmentedLargeMultinomialLogitStep

urbansim_templates.__version__

'0.1.dev14'

### Set up testing data

In [2]:
d1 = {'oid': np.arange(10), 
      'group': np.random.choice(['A','B','C'], size=10),
      'obsval': np.random.random(10),
      'choice': np.random.choice(np.arange(20), size=10)}

d2 = {'aid': np.arange(20), 
      'altval': np.random.random(20)}

obs = pd.DataFrame(d1).set_index('oid')
orca.add_table('obs', obs)

alts = pd.DataFrame(d2).set_index('aid')
orca.add_table('alts', alts)

In [3]:
obs

,group,obsval,choice
oid,,,
0,B,0.406087,13
1,A,0.888636,3
2,A,0.046671,6
3,A,0.618342,0
4,A,0.635820,11
5,A,0.612325,8
6,C,0.658141,9
7,B,0.076418,11
8,C,0.336237,17


### Test model features

In [4]:
m = SegmentedLargeMultinomialLogitStep()

In [5]:
m.defaults.choosers = 'obs'
m.defaults.alternatives = 'alts'
m.defaults.choice_column = 'choice'
m.defaults.model_expression = 'obsval + altval'
m.segmentation_column = 'group'

In [6]:
m.build_submodels()

Building submodels for 3 categories: ['A' 'B' 'C']


In [7]:
for k, v in m.submodels.items():
    print(v.chooser_filters)

group == 'A'
group == 'B'
group == 'C'


In [8]:
# Change one model expression

m.submodels['A'].model_expression = 'altval'

for k, v in m.submodels.items():
    print(v.model_expression)

altval
obsval + altval
obsval + altval


In [9]:
# Reset the model expressions

m.defaults.model_expression = 'obsval + altval'

for k, v in m.submodels.items():
    print(v.model_expression)

obsval + altval
obsval + altval
obsval + altval


In [10]:
m.fit_all()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:              5
Model:         Multinomial Logit   Df Residuals:                  2
Method:       Maximum Likelihood   Df Model:                      3
Date:                 2018-09-25   Pseudo R-squ.:             0.025
Time:                      11:07   Pseudo R-bar-squ.:        -0.179
AIC:                      34.703   Log-Likelihood:          -14.352
BIC:                      33.532   LL-Null:                 -14.722
               coef   std err         z     P>|z|   Conf. Int.
--------------------------------------------------------------
Intercept   -0.0000     1.227    -0.000     1.000             
obsval      -0.0000     1.681    -0.000     1.000             
altval      -1.4399     1.644    -0.876     0.381             
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:              2
Model

/Users/maurer/Dropbox/Git-imac/udst/choicemodels/choicemodels/mnl.py:235: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  chosen = np.reshape(self._df[[self._choice_col]].as_matrix(),
/Users/maurer/Dropbox/Git-imac/udst/choicemodels/choicemodels/mnl.py:238: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  log_lik, fit = mnl_estimate(model_design.as_matrix(), chosen, self._numalts)


In [11]:
m.to_dict()

{'template': 'SegmentedLargeMultinomialLogitStep',
 'template_version': '0.1.dev14',
 'name': 'SegmentedLargeMultinomialLogitStep-20180925-110705',
 'tags': [],
 'defaults': {'template': 'LargeMultinomialLogitStep',
  'template_version': '0.1.dev14',
  'name': None,
  'tags': [],
  'choosers': 'obs',
  'alternatives': 'alts',
  'model_expression': 'obsval + altval',
  'choice_column': 'choice',
  'chooser_filters': None,
  'chooser_sample_size': None,
  'alt_filters': None,
  'alt_sample_size': None,
  'out_choosers': None,
  'out_alternatives': None,
  'out_column': None,
  'out_chooser_filters': None,
  'out_alt_filters': None,
  'summary_table': None,
  'fitted_parameters': None},
 'segmentation_column': 'group',
 'submodels': {'A': {'template': 'LargeMultinomialLogitStep',
   'template_version': '0.1.dev14',
   'name': 'LargeMultinomialLogitStep-20180925-110705',
   'tags': [],
   'choosers': 'obs',
   'alternatives': 'alts',
   'model_expression': 'obsval + altval',
   'choice_col

In [12]:
modelmanager.initialize()

Registering model step 'test1'
Registering model step 'SegmentedLargeMultinomialLogitStep-20180925-104951'
Registering model step 'SegmentedLargeMultinomialLogitStep-20180925-103111'
Registering model step 'test2'


In [13]:
modelmanager.register(m)

Saving 'SegmentedLargeMultinomialLogitStep-20180925-110705.yaml': /Users/maurer/Dropbox/Git-imac/udst/public-template-workspace/notebooks-sam/configs
Registering model step 'SegmentedLargeMultinomialLogitStep-20180925-110705'


In [14]:
m.name = 'test1'
modelmanager.register(m)
m2 = modelmanager.get_step('test1')
m2.name = 'test2'
modelmanager.register(m2)

Saving 'test1.yaml': /Users/maurer/Dropbox/Git-imac/udst/public-template-workspace/notebooks-sam/configs
Registering model step 'test1'
Saving 'test2.yaml': /Users/maurer/Dropbox/Git-imac/udst/public-template-workspace/notebooks-sam/configs
Registering model step 'test2'


In [15]:
m.defaults.chooser_filters = ['test', '2']